# Bienvenido a la Jupyter Notebook de ejecución de visualizaciones


**(!)WARNING(!)**
**Condición necesaria para que el código funcione correctamente:**
**El archivo csv de lectura debe estar escrito todo en minúsculas.**

In [2]:
# Librerías Nativas de Python
import pickle, os

# Librerías de terceros
import pandas as pd
from dotenv import load_dotenv
from sklearn.preprocessing import OneHotEncoder
# import chart_studio.plotly as py
# import chart_studio.tools as tls

# Librería RD
from tools.feed import data_info
from tools.feature_treatment import resample_dataset

load_dotenv()

pd.options.plotting.backend = "plotly"

# USER = os.environ["USER_CS"]
# TOKEN = os.environ["TOKEN_CS"]
# tls.set_credentials_file(username=USER, api_key=TOKEN)

In [2]:
print(os.listdir("./app/shared_resources/"))

['dfts_carne_total_precio_10-22_1.pickle', 'dfts_data-benchmark_10-21_1.pickle', 'dfts_data-benchmark_10-22_1.pickle', 'dfts_melcogatev2_10-21_1.pickle', 'dfts_melcogatev2_10-21_2.pickle', 'dfts_melcogatev2_10-22_1.pickle']


In [3]:
# APP
from app.time_series import TimeSeries

file_name = "hola mundo"
file_name = "dfts_melcogatev2_10-21_1.pickle"


if file_name not in os.listdir("./app/shared_resources/"):
    TimeSeries.main()


##  cargar datos en vez de generarlos ya que podriamos haberlo corrido anteriormente
else:
    with open(f"./app/shared_resources/{file_name}", "rb") as file:
        df = pickle.load(file)

In [4]:
# with open("./app/shared_resources/dfts_melcogatev2_10_20_1.pickle", "rb") as file:
#     df = pickle.load(file)

In [5]:
df.head(3)

,@timestamp,@version,_id,_index,_score,agent,author,category,content,coordinates,...,url.domain,url.path,user_agent.original,user_followers,username,usuarios_mencionados,hashtags,datetime,date_str,time_str
0,"Oct 18, 2023 @ 10:40:00.000",1,twt_1714637633782005873,in-politicaargentina3-2023-10,-,twt-brow,AlbornJuan,Twitter,no vi ninguno en las ramas -q soberano soberan...,-,...,dashboard.reputacion.digital,/ingest,python-requests/2.31.0,"1,116",Juan leonardo Alborn,"[@contracurva, @fercerimedo_ok]",NaN,2023-10-18 10:40:00,2023-10-18,10:40:00
1,"Oct 18, 2023 @ 10:34:00.000",1,twt_1714636000226349218,in-politicaargentina7-2023-10,-,twt-brow,NicoteFornaso,Twitter,jajajajaja más para entre ríos??? más para mee...,-,...,dashboard.reputacion.digital,/ingest,python-requests/2.31.0,471,Nicote,"[@eleccionesdatos, @adanhbahl]","[#melcogate, #insurraldegate, #tetazgate]",2023-10-18 10:34:00,2023-10-18,10:34:00
2,"Oct 18, 2023 @ 10:22:00.000",1,twt_1714633151576727671,in-politicaargentina5-2023-10,-,twt-brow,frauleinmaria,Twitter,y ni hablar de la complicidad de bullrich con ...,-,...,dashboard.reputacion.digital,/ingest,python-requests/2.31.0,"4,312",Graciela ✨,NaN,[#melconian],2023-10-18 10:22:00,2023-10-18,10:22:00


In [6]:
from app.m1_sentiment.M1_sentiment import main_shared_resources

max_workers = 4
# max_workers = 1 # 4 minutos
# max_workers = 4 # 3 minutos y medio
# max_workers = 5 # 3 minutos

df = main_shared_resources(file_name, max_workers)
df = df.drop(columns = ["spark_sentiment", "spark_emotion"])


Cargando configuración...
Leyendo fichero...
Fichero leído con éxito.

Ejecutando modelo optimizado con 4 hilos...
Modelo - clasificación de sentimientos: cardiffnlp/twitter-xlm-roberta-base-sentiment
Tiempo de ejecución del modelo:  222.17151284217834
Ensamble de las predicciones
Predicción de sentimientos finalizada.


In [7]:
df.sentiment_i.to_frame().isna().sum()
df.score_sentiment_i.to_frame().isna().sum()
df.score_sentiment_i.to_frame()

,score_sentiment_i
0,0.548634
1,0.819689
2,0.947779
3,0.923712
4,0.516778
...,...
6911,0.604728
6912,0.526588
6913,0.933041
6914,0.934070


# Feature Selection

**(!)WARNING(!)**

**El indice de confiabilidad de spark_emotion y spark_sentiment es nulo, por lo que se remueven dichas variables y se regeneran dichas features.**

#### Features para el análisis de contexto

Cuando se quiere identificar qué es lo que está acontenciendo en determinado período de tiempo.
<br />
<center><b>

```
[datetime, content, token, username, usuarios_mencionados, hashtags, date_str, time_str, category]

[datetime, content, len(usuario_mencionado), len(hastags), spark_emotion, spark_sentiment, category]
```

</b></center>
<br />

#### Features para análisis de las conversaciones

Cuando se quiere identificar el fuerte de las conversaciones alrededor de un perfil determinado, es necesario conocer las fuentes que participan.

<br />
<center><b>

```


[quote_count, reach, replay_count, retweet_count, rt_like_count, rt_quote_count, rt_reply_count, rt_retweet_count, rt_text]

[category, type, author,  source, description_user, [favorite_count, friends_count, user_followers], [is_quoted, is_reply, is_retweet]]
 
```

</b></center>
<br />

 

In [8]:
## TEST 1
features = ["datetime", "content", "token", "usuarios_mencionados", "hashtags", "username", "date_str", "time_str", "category"]

for feature in features:
    if feature not in df.columns:
        raise Exception(f"Faltan features en la data recibida:\nFeature faltante \"{feature}\"")

In [9]:
# Obtenemos features e invertimos el orden del dataset
dataset = df.loc[::-1, features]

In [10]:
dataset.head(10)

,datetime,content,token,usuarios_mencionados,hashtags,username,date_str,time_str,category
6915,2023-10-15 13:17:00,la gran pregunta si estos audios surgieron de ...,"[gran, pregunta, audios, surgieron, investigac...","[@marianschw93348, @eligiria, @edufeiok]",NaN,Maria Sil💙⭐⭐⭐,2023-10-15,13:17:00,Twitter
6914,2023-10-15 13:18:00,por que los votantes trolls de pb no quieren a...,"[votantes, trolls, quieren, admitir, chanta, m...",[@robertojg1810],NaN,◇Carolina Reaper◇,2023-10-15,13:18:00,Twitter
6913,2023-10-15 13:18:00,julito como te va? bullrich se esta hundiendo ...,"[julito, bullrich, hundiendo, solita, audios, ...","[@jdgoldestein, @patobullrich, @jmilei, @llibe...",NaN,Patricia Mollo,2023-10-15,13:18:00,Twitter
6912,2023-10-15 13:18:00,"los periodistas , no hablan de los audios de m...","[periodistas, hablan, audios, melconia]",[@peponila],NaN,Ofe,2023-10-15,13:18:00,Twitter
6911,2023-10-15 13:18:00,le hubiese convenido mas quedarse comiendo piz...,"[convenido, quedarse, comiendo, pizza, casa, p...",[@gonziver],NaN,The Boogeyman,2023-10-15,13:18:00,Twitter
6910,2023-10-15 13:18:00,"no puede ser tan hijo de mil vecinos , como va...","[hijo, mil, vecinos, vida, privada, nadie, hab...",[@jonatanviale],NaN,Bautista V,2023-10-15,13:18:00,Twitter
6909,2023-10-15 13:18:00,? a que se debe esa conclusion? a lanata le es...,"[debe, conclusion, lanata, exigiendo, hable, m...","[@fperisset, @tata68276236]",NaN,Petula ll,2023-10-15,13:18:00,Twitter
6908,2023-10-15 13:19:00,increiblemente este tuit termino siendo el res...,"[increiblemente, tuit, termino, resumen, expli...",NaN,NaN,Mau Grande,2023-10-15,13:19:00,Twitter
6907,2023-10-15 13:19:00,yo lo veo como q el votante de larreta ya deci...,"[veo, votante, larreta, decidio, voto, jxc, es...",[@esquizonomics],NaN,odio el rkt,2023-10-15,13:19:00,Twitter
6906,2023-10-15 13:19:00,mirtha le dio una paliza monumental al viejo v...,"[mirtha, dio, paliza, monumental, viejo, verde...",[@amazonaarg],[#melcogate],Th3Crak47OMG...,2023-10-15,13:19:00,Twitter


**Obtención del dataset de procesamiento**

**IMPORTANTE**

**Cuando trabajamos con series temporales, las primeras filas son los datos más antiguos y las últimas filas son los datos más nuevos.**

**Cosa que si nos llegan datos nuevos, se los agregamos al final.**

In [11]:
## TEST 2
features = ["datetime", "content", "sentiment_i", "score_sentiment_i"]# spark_emotion
for feature in features:
    if feature not in df.columns:
        raise Exception(f"Faltan features en la data recibida:\nFeature faltante \"{feature}\"")
# pendiente: , len(usuario_mencionado), len(hastags)
# pendiente: token

In [12]:
# Seleccionamos features e invertimos el orden del dataset
dataset_main = df.loc[::-1, features]

# dataset_main["spark_emotion"] = [elemento if elemento != '-' else None for elemento in dataset_main.spark_emotion.to_list()]
# dataset_main["spark_sentiment"] = [elemento if elemento != '-' else None for elemento in dataset_main.spark_sentiment.to_list()]

dataset_main

,datetime,content,sentiment_i,score_sentiment_i
6915,2023-10-15 13:17:00,la gran pregunta si estos audios surgieron de ...,negative,0.906671
6914,2023-10-15 13:18:00,por que los votantes trolls de pb no quieren a...,negative,0.934070
6913,2023-10-15 13:18:00,julito como te va? bullrich se esta hundiendo ...,negative,0.933041
6912,2023-10-15 13:18:00,"los periodistas , no hablan de los audios de m...",neutral,0.526588
6911,2023-10-15 13:18:00,le hubiese convenido mas quedarse comiendo piz...,negative,0.604728
...,...,...,...,...
4,2023-10-18 10:18:00,publico y notorio la complicidad de bullrich c...,neutral,0.516778
3,2023-10-18 10:22:00,en serio crees que muchos votos que fueron par...,negative,0.923712
2,2023-10-18 10:22:00,y ni hablar de la complicidad de bullrich con ...,negative,0.947779
1,2023-10-18 10:34:00,jajajajaja mas para entre rios??? mas para mee...,negative,0.819689


In [13]:
data_info(dataset_main)

,columna,Nan,pct_nan,dtype,count,pct_reg,count_unique
0,datetime,0,0.0,datetime64[ns],6916,100.0,2685
1,content,0,0.0,object,6916,100.0,6914
2,sentiment_i,0,0.0,object,6916,100.0,3
3,score_sentiment_i,0,0.0,float64,6916,100.0,1729


In [14]:
## implementar features, cantidad de hastags, cantidad de usuarios mencionados

# Visualizaciones

**Frecuencias de los Marcos Temporales**

T: minuto <br />
H: hora <br />
D: día <br />
W: semana <br />
M: mes <br />
 <br />
5T = 5 minutos <br />
2D = 2 días <br />

In [15]:
# Funciones muy útiles para discretizar variables

# category_emotion, label_emotion = pd.factorize(df['spark_emotion'])
# category_sentiment, label_sentiment = pd.factorize(df['spark_sentiment'])

In [16]:
## [ORIGINAL]
# mask = dataset_main.spark_sentiment.notna()

# # f: feature, m: muestras y n: dimensión de las features
# f1 = dataset_main.loc[mask, :]
# ## Incorporar features generadas por dataset


# # Crear una instancia de OneHotEncoder
# encoder_i = OneHotEncoder(sparse_output=False)
# encoder_ii = OneHotEncoder(sparse_output=False)

# # Ajustar y transformar los datos
# category_sentiment_encoded = encoder_i.fit_transform(f1[['spark_sentiment']])
# category_emotion_encoded = encoder_ii.fit_transform(f1[['spark_emotion']])

# # Convertir a DataFrame y unirlo al DataFrame original
# sentiment_encoded = pd.DataFrame(category_sentiment_encoded, columns=encoder_i.get_feature_names_out(['spark_sentiment']), index=f1.index)
# emotion_encoded = pd.DataFrame(category_emotion_encoded, columns=encoder_ii.get_feature_names_out(['spark_emotion']), index=f1.index)

# f1 = pd.concat([f1, sentiment_encoded, emotion_encoded], axis=1)


# # Establecer 'datetime' como índice
# f1 = f1.set_index('datetime')
# # f1 = f1.resample('60T').asfreq()
# # Listado de las columnas que no son numéricas
# non_numeric_column = data_info(f1).loc[(data_info(f1)["dtype"] != float).values].columna.values

# f1_categorical = f1[non_numeric_column]
# f1_numerical = f1.drop(non_numeric_column, axis=1)

In [17]:
f1 = dataset_main.copy()

# Crear una instancia de OneHotEncoder
encoder_i = OneHotEncoder(sparse_output=False)
# encoder_ii = OneHotEncoder(sparse_output=False)

# Ajustar y transformar los datos
category_sentiment_encoded = encoder_i.fit_transform(f1[['sentiment_i']])
# category_emotion_encoded = encoder_ii.fit_transform(f1[['spark_emotion']])

# Convertir a DataFrame y unirlo al DataFrame original
sentiment_encoded = pd.DataFrame(category_sentiment_encoded, columns=encoder_i.get_feature_names_out(['sentiment_i']), index=f1.index)
# emotion_encoded = pd.DataFrame(category_emotion_encoded, columns=encoder_ii.get_feature_names_out(['spark_emotion']), index=f1.index)

# f1 = pd.concat([f1, sentiment_encoded, emotion_encoded], axis=1)
f1 = pd.concat([f1, sentiment_encoded], axis=1)


# Establecer 'datetime' como índice
f1 = f1.set_index('datetime')

f1.head(5)

In [ ]:
f1_15min = resample_dataset(f1, '15T')
f1_60min = resample_dataset(f1, '1H')

### Resampleo de la muestra

In [18]:
# freq = '4H'
freq = '60T'


    
    
f1_categorical = f1_categorical.resample(freq).agg(lambda x: list(x))
f1_numerical = f1_numerical.resample(freq).mean()

f1_resampled = pd.concat([f1_categorical, f1_numerical], axis=1)

f1_resampled.head(5)

,content,sentiment_i,score_sentiment_i,sentiment_i_negative,sentiment_i_neutral,sentiment_i_positive
datetime,,,,,,
2023-10-15 13:00:00,[la gran pregunta si estos audios surgieron de...,"[negative, negative, negative, neutral, negati...",0.778229,0.813793,0.151724,0.034483
2023-10-15 14:00:00,[ya repudio pitta las recomendaciones de melco...,"[negative, negative, negative, negative, negat...",0.744067,0.730469,0.238281,0.031250
2023-10-15 15:00:00,[y de melconian no tenes nada para decir? se n...,"[negative, neutral, negative, negative, negati...",0.750176,0.746154,0.203846,0.050000
2023-10-15 16:00:00,[filtraron polemicos audios de carlos melconia...,"[negative, negative, negative, neutral, neutra...",0.744609,0.764192,0.179039,0.056769
2023-10-15 17:00:00,[eso es un delirio mistico? no te parece una c...,"[negative, negative, negative, negative, negat...",0.757811,0.766667,0.171429,0.061905


In [19]:
f1_visual = f1_resampled.filter(regex='^sentiment_i').fillna(0.)
non_numeric_column = data_info(f1_visual).loc[(data_info(f1_visual)["dtype"] != float).values].columna.values
f1_visual = f1_visual.drop(non_numeric_column, axis=1)


visual_kwargs = {
    "width": 1200,
    "height": 600,
    "template": "plotly_dark",
    "title": f"Linea de tiempo de evolución de los sentimiento | Marco Temporal: {freq}",
    # "labels": {"variable": "Emociones", "value": "Porcentaje de emociones", "label": "Emociones"},
    # "color": "label",
    # "color_discrete_map": colores
    }


figure = f1_visual.plot(kind="line", **visual_kwargs).update_layout(showlegend=True)

f1_visual

c:\Users\Agustin\Desktop\RD\Octubre\RD_APP\venv\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


,sentiment_i_negative,sentiment_i_neutral,sentiment_i_positive
datetime,,,
2023-10-15 13:00:00,0.813793,0.151724,0.034483
2023-10-15 14:00:00,0.730469,0.238281,0.031250
2023-10-15 15:00:00,0.746154,0.203846,0.050000
2023-10-15 16:00:00,0.764192,0.179039,0.056769
2023-10-15 17:00:00,0.766667,0.171429,0.061905
...,...,...,...
2023-10-18 06:00:00,0.756098,0.219512,0.024390
2023-10-18 07:00:00,0.752941,0.200000,0.047059
2023-10-18 08:00:00,0.716129,0.206452,0.077419


In [20]:
figure.show()
